In [1]:
# Day 21 --Allergen Assessment--
from collections import Counter

def initialize(data):
    allergen_ingredient_map = dict()
    ingredient_counts = Counter()
    for line in data:
        ingredients, allergens = line.split(' (contains ')
        ingredients = set(ingredients.split(' '))
        allergens = allergens.strip(')').split(', ')
        for i in ingredients:
            ingredient_counts[i] += 1
        for a in allergens:
            if a in allergen_ingredient_map:
                allergen_ingredient_map[a] = allergen_ingredient_map[a].intersection(ingredients)
            else:
                allergen_ingredient_map[a] = ingredients
    return allergen_ingredient_map, ingredient_counts

def get_safe_count(allergen_ingredient_map, ingredient_counts):
    all_poss_allergens = set().union(*allergen_ingredient_map.values())
    safe_count = 0
    for i in ingredient_counts:
        if i not in all_poss_allergens:
            safe_count += ingredient_counts[i]
    return safe_count

def reduce_map(allergen_ingredient_map):
    committed = set()
    finished = False
    while not finished:
        finished = True
        for allergen in allergen_ingredient_map:
            poss_ingredients = allergen_ingredient_map[allergen]
            if len(poss_ingredients) == 1:
                committed.add(next(iter(poss_ingredients)))
            else:
                finished = False
                for ingredient in list(poss_ingredients):
                    if ingredient in committed:
                        poss_ingredients.remove(ingredient)
                allergen_ingredient_map[allergen] = poss_ingredients
    return allergen_ingredient_map

def run(my_file):
    data = [x.strip() for x in open(my_file)]
    #(1)wittle down possible ingredients containing each allergen (2)count #occurences each ingredient
    allergen_ingredient_map, ingredient_counts = initialize(data)
    #count total occurences of ingredients so far known to be safe (pt1 ans)
    safe_count = get_safe_count(allergen_ingredient_map, ingredient_counts)
    #reduce allergen_ingredient_map until it is one-to-one
    allergen_ingredient_map = reduce_map(allergen_ingredient_map)
    canonical_dil = ''
    for a in sorted(allergen_ingredient_map):
        ingredient = next(iter(allergen_ingredient_map[a]))
        canonical_dil += (ingredient + ',')            
    return safe_count, canonical_dil[:-1]
    
my_file = 'test.txt'
my_file = 'day_21.txt'
pt1, pt2 = run(my_file)
print(f'pt1: {pt1}, pt2: {pt2}')

In [2]:
# Day 21 -- slightly better data struct, but actually longer
def clean(data):
    food = list()
    for line in data:
        ingredients, allergens = line.split(' (contains ')
        ingredients = ingredients.split(' ')
        allergens = allergens.strip(')').split(', ')
        food.append((ingredients, allergens))
    return food

def get_a_i_pairs(food: list(tuple((list, list)))):
    # 'a' means allergens, 'i' means ingredients
    all_a = set()
    all_i = set()
    for i, a in food:
        all_a = all_a.union(set(a))
        all_i = all_i.union(set(i))
        
    a_i_pairs = [] #each (allergen, set of possible ingredients containing it)
    for allergen in all_a:
        poss_ingredients = set()
        for i, a in food:
            if allergen in a:
                poss_ingredients = poss_ingredients.intersection(i) if poss_ingredients else set(i)
        a_i_pairs.append((allergen, poss_ingredients))
    return a_i_pairs

def get_safe_count(food, a_i_pairs):
    all_poss_allergens = set().union(*[pair[1] for pair in a_i_pairs])
    safe_count = 0
    for ingredients, _ in food:
        for i in ingredients:
            if i not in all_poss_allergens:
                safe_count += 1
    return safe_count

def reduce_pairs(a_i_pairs):
    committed = set()
    finished = False
    while not finished:
        finished = True
        for allergen, poss_ingredients in a_i_pairs:
            if len(poss_ingredients) == 1:
                committed.add(next(iter(poss_ingredients)))
            else:
                finished = False
                for ingredient in list(poss_ingredients):
                    if ingredient in committed:
                        poss_ingredients.remove(ingredient)
    return a_i_pairs

def run(my_file):
    data = [x.strip() for x in open(my_file)]
    food = clean(data) #list of tuples: (ingredient_list, allergen_list) for each food in data
    a_i_pairs = get_a_i_pairs(food) #list of tuples: (allergen, possible ingredients containing that allergen)
    #count total occurences of ingredients so far known to be safe (pt1 ans)
    safe_count = get_safe_count(food, a_i_pairs)
    #reduce allergen/ingredient pairs until they are one-to-one
    a_i_pairs = reduce_pairs(a_i_pairs) 
    canonical_dil = ''
    for _, i in sorted(a_i_pairs):
        canonical_dil += (next(iter(i)) + ',')            
    return safe_count, canonical_dil[:-1]
    
my_file = 'test.txt'
my_file = 'day_21.txt'
pt1, pt2 = run(my_file)
print(f'pt1: {pt1}, pt2: {pt2}')

In [32]:
#Day 22 --Crab Combat--
from collections import deque
from itertools import islice

def extend_round_winner(p1, p2, card1, card2, winner_bool):
    if winner_bool: #evaluates to True if p1 won the round
        p1.extend([card1, card2])
    else:
        p2.extend([card2, card1])
    return p1, p2

def play_combat(p1, p2):
    while p1 and p2:
        card1, card2 = p1.popleft(), p2.popleft()
        p1, p2 = extend_round_winner(p1, p2, card1, card2, card1>card2)
    return p1 if p1 else p2

def play_recursive_combat(p1, p2):
    used_p1, used_p2 = set(), set()

    while p1 and p2:
        check_p1, check_p2 = tuple(p1), tuple(p2)
        if check_p1 in used_p1 or check_p2 in used_p2:
            return (p1, True) #if stalemate (inf. loop), award p1 victory
        else:
            used_p1.add(check_p1), used_p2.add(check_p2)

        card1, card2 = p1.popleft(), p2.popleft()
        winner_bool = None
        if card1 <= len(p1) and card2 <= len(p2):
            p1_rec_deck = deque(islice(p1, 0, card1))
            p2_rec_deck = deque(islice(p2, 0, card2))
            #since stalemates go to p1, if p1 has highest card she can't lose
            if max(p1_rec_deck) > max(p2_rec_deck):
                winner_bool = True
            else: #if p2 has highest card, play out a sub-game and return this round's winner (element 1)
                winner_bool = play_recursive_combat(p1_rec_deck, p2_rec_deck)[1]

        if winner_bool != None:
            p1, p2 = extend_round_winner(p1, p2, card1, card2, winner_bool)
        else:
            p1, p2 = extend_round_winner(p1, p2, card1, card2, card1>card2)

    return (p1, True) if p1 else (p2, False)

def count_score(winning_deck):
    score = 0
    for i, card in enumerate(reversed(winning_deck), start=1):
        score += card * i
    return score

def run(my_file):
    #create a deque container for each player
    players = open(my_file).read().strip().split('\n\n')
    p1, p2 = players[0].split('\n'), players[1].split('\n')
    p1, p2 = deque(map(int, p1[1:])), deque(map(int, p2[1:]))
#     print(p1, p2)
    #play game until finished
    winner1 = play_combat(p1.copy(), p2.copy())
    score1= count_score(winner1)
    winner2, _ = play_recursive_combat(p1.copy(), p2.copy())
    score2= count_score(winner2)
    return score1, score2
    
    
my_file = 'test.txt'
my_file = 'day_22.txt'
pt1, pt2 = run(my_file)
print(f'pt1: {pt1}, pt2: {pt2}')

pt1: 33403, pt2: 29177


In [33]:
# Day 23 -- Crab Cups -- runs in 12s
def get_label_wise_neighbours(input_, curr, ispt2):
    """find the (clockwise) neighbour of each cup"""
    #append the neighbour for the right-most cup in our input
    input_ = input_ + [10] if ispt2 else input_ + [curr]
    #generate a list of [cup 1's neighb,..., cup 9's neighb]
    neighbs = [69] + [input_[input_.index(i)+1] for i in range(1,10)] #69 is a dummy var bc no cup labeled zero
    if ispt2: #complete neighbours for massive pt2 array
        neighbs += list(range(11, 1000001)) + [curr]
    return neighbs

def sim_crab(neighbs, curr, num_moves, max_val):
    # print(neighbs[:20], curr, num_moves, max_val, len(neighbs), neighbs[-2:])
    for i in range(num_moves):
        this_cup = curr
        three_cups = list()
        for _ in range(3): #find three closest neighbours of current_cup, so we can move them
            this_cup = neighbs[this_cup]
            three_cups += [this_cup]
        #destination cup: subtract one until hit eligible cup, else just take the largest label we have
        dest = curr-1 if curr > 1 else max_val
        while dest in three_cups:
            dest = dest-1 if dest > 1 else max_val
        #what was formerly the neighbour of the third cup we picked up will now slide in next to curr_cup, and thus become curr_cup next time
        next_ = neighbs[this_cup]
        #the new neighbour of said third cup now becomes what was previously the neighbour of our dest_cup (since we slid in front of it)
        neighbs[this_cup] = neighbs[dest]
        #the new neighbour of dest_cup becomes what was formerly the neighbour of curr_cup, i.e. the first of the three we moved
        neighbs[dest] = neighbs[curr]
        #as established previously, 'next_' is the cup which slid in next to curr, it is now okay to override
        neighbs[curr] = next_
        curr = next_
    return neighbs

def process_result(neighbs, ispt2):
    if ispt2:
        next_to_one = neighbs[1]
        next_to_that = neighbs[next_to_one]
        print(next_to_one, next_to_that)
        return next_to_one*next_to_that
    else:
        to_print = list()
        x = 1
        for _ in range(8):
            x = neighbs[x]
            to_print.append(x)
        return ''.join(map(str, to_print))

def run(my_input, ispt2):
    input_ = [int(c) for c in my_input]
    curr = input_[0] #current cup
    neighbs = get_label_wise_neighbours(input_, curr, ispt2)
    num_moves = 10000000 if ispt2 else 100
    max_val = 1000000 if ispt2 else 9
    neighbs = sim_crab(neighbs, curr, num_moves, max_val)
    return process_result(neighbs, ispt2)
    
    
my_input = '389125467' #test
my_input = '792845136' #real
pt1, pt2 = run(my_input, False), run(my_input, True)
print(f'pt1: {pt1}, pt2: {pt2}')

404237 728089
pt1: 98742365, pt2: 294320513093


In [34]:
# Day 24 -- Lobby Layout -- runs in ~18s
from collections import Counter

def parse_tiles(tiles):
    d = 3**.5 #(d = y_dist) if a straight e/w move is two units, a diag move must be one unit e/w, sqrt(3) units in the n/s direction
    direction_map = {'ne':(1,1), 'se':(1,-1), 'sw':(-1,-1), 'nw':(-1,1), 'e':(2,0), 'w':(-2,0)}
    tile_counts = Counter()
    for tile in tiles:
        x, y, idx = 0, 0, 0
        while idx < len(tile):
            if tile[idx:idx+2] in direction_map:
                move = direction_map[tile[idx:idx+2]]
                x += move[0]
                y += move[1]
                idx += 2
            else:
                move = direction_map[tile[idx:idx+1]]
                x += move[0]
                y += move[1]
                idx += 1
        tile_counts[(x,y)] += 1
    return tile_counts

def count_black(tile_counts):
    black_tiles = 0
    for t in tile_counts:
        if tile_counts[t]%2 == 1:
            black_tiles += 1
    return black_tiles

def sim(tile_counts, days=1):
    transforms = [(1,1), (1,-1), (-1,-1), (-1,1), (2,0), (-2,0)]
    for d in range(1, days+1):
        b_neighbors = Counter() #count num black tiles next to any given tile
        black_now = [t for t in tile_counts if tile_counts[t]%2 == 1]
        for tile in black_now:
            x, y = tile[0], tile[1]
            for t in transforms:
                a, b = x + t[0], y + t[1]
                b_neighbors[(a,b)] += 1
        for t in black_now:
            if (b_neighbors[t] == 0) or (b_neighbors[t] > 2):
                tile_counts[t] += 1 #flip to white
        for w in b_neighbors:
            if w not in black_now and b_neighbors[w] == 2:
                tile_counts[w] += 1 #flip to black
    return tile_counts

def run(my_file):
    tiles = open(my_file).read().strip().split('\n')
    tile_counts = parse_tiles(tiles)
    tile_counts2 = sim(tile_counts.copy(), days=100)
    return count_black(tile_counts), count_black(tile_counts2)
    
my_file = 'test.txt'
my_file = 'day_24.txt'
pt1, pt2 = run(my_file)
print(f'pt1: {pt1}, pt2: {pt2}')

pt1: 330, pt2: 3711


In [35]:
# Day 25 -- Combo Breaker -- runs in ~25s...
def transform(z=1, subj=7, divisor=20201227, n=1):
    for _ in range(n):
        z *= subj #step one
        z = z%divisor #step two
    return z

def get_loop_size(pub_key, subj=7, divisor=20201227):
    z, loops = 1, 0
    while z!= pub_key and loops < 1000000000:
        loops += 1
        z = transform(z, subj, divisor, n=1)
    return loops

def run(card, door):
    c_loop, d_loop = get_loop_size(card, subj=7), get_loop_size(door, subj=7)
#     print(c_loop, d_loop)
    encryption_key = transform(subj=card, n=d_loop)
    # print(transform(subj=door, n=c_loop))
    return encryption_key
    
CARD, DOOR = 5764801, 17807724 #Test
CARD, DOOR = 11562782, 18108497 #Real
print(run(CARD, DOOR))

17580934 19976408
2947148
